<a href="https://colab.research.google.com/github/Sidhtang/facial-recognition-with-open-cv-and-cnn/blob/main/Copy_of_facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import cv2
from keras.models import model_from_json
import numpy as np

In [ ]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git

fatal: destination path 'facial_expressions' already exists and is not an empty directory.


In [ ]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
#for each emotion we will create a subdirectory
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/training', emotion, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/testing', emotion, image)
    copyfile(source, dest)

NameError: name 'os' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(8, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 64)        0

In [ ]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [ ]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [ ]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Epoch 1/10


<ipython-input-8-a67ff5996604>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


86/86 [==============================] - ETA: 0s - loss: 1.6193 - accuracy: 0.4726

86/86 [==============================] - 21s 200ms/step - loss: 1.6193 - accuracy: 0.4726 - val_loss: 1.0527 - val_accuracy: 0.5011
Epoch 2/10
86/86 [==============================] - ETA: 0s - loss: 1.0436 - accuracy: 0.4923

86/86 [==============================] - 15s 179ms/step - loss: 1.0436 - accuracy: 0.4923 - val_loss: 1.0482 - val_accuracy: 0.5011
Epoch 3/10
86/86 [==============================] - ETA: 0s - loss: 1.0438 - accuracy: 0.4971

...

86/86 [==============================] - 15s 174ms/step - loss: 1.0393 - accuracy: 0.5018 - val_loss: 1.0426 - val_accuracy: 0.5011
Epoch 7/10
86/86 [==============================] - ETA: 0s - loss: 1.0403 - accuracy: 0.4980

86/86 [==============================] - 18s 210ms/step - loss: 1.0403 - accuracy: 0.4980 - val_loss: 1.0446 - val_accuracy: 0.5011
Epoch 8/10
86/86 [==============================] - ETA: 0s - loss: 1.0395 - accuracy: 0.5018

86/86 [==============================] - 15s 178ms/step - loss: 1.0395 - accuracy: 0.5018 - val_loss: 1.0421 - val_accuracy: 0.5011
Epoch 9/10
86/86 [==============================] - ETA: 0s - loss: 1.0399 - accuracy: 0.5018

86/86 [==============================] - 15s 171ms/step - loss: 1.0399 - accuracy: 0.5018 - val_loss: 1.0440 - val_accuracy: 0.5011
Epoch 10/10
86/86 [==============================] - ETA: 0s - loss: 1.0411 - accuracy: 0.4935

86/86 [==============================] - 17s 202ms/step - loss: 1.0411 - accuracy: 0.4935 - val_loss: 1.0476 - val_accuracy: 0.5011


In [ ]:
   model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
model.save_weights('model.h5')

In [ ]:

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful",
                3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model.load_weights("model.h5")

In [ ]:
import cv2
import numpy as np

# Assuming the model and emotion_dict are defined somewhere
# model = ...
# emotion_dict = ...

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error opening video stream or file")
    exit()  # Exit if the video capture fails

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:  # Check if a frame was successfully read
        print("Can't receive frame (stream end?). Exiting ...")
        break

    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        print(ret)
        break

    # Create a face detector
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # Take each face available on the camera and preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # Predict emotion
        emotion_prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))

        # Put text of emotion on the frame
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Error opening video stream or file
Can't receive frame (stream end?). Exiting ...
